## Sara the Turtle: `Pandas` Redux
Recalling the script we wrote earlier to process the ARGOS tracking data for Sara the Turtle. Here, we'll demonstrate how another 3rd party package, namely `pandas`, can greatly simplify that task.

In [1]:
#Set the user date (can be switched to be a user input...)
userDate = '7/3/2003'

In [2]:
#Set the location classes used to filter the data. Again, can be an input
locClasses = '1','2','3'

In [3]:
#import the modules
import pandas as pd

In [4]:
#set a variable to the path where the tracking data lives
dataFilename = '..\Data\Sara.txt'

In [5]:
#Open the data as a pandas dataframe. 
df = pd.read_csv(dataFilename,
                 comment='#',   #Skip lines that start with '#'
                 delimiter='\t' #Set the delimiter as <tab>
                )
df

,uid,tag_id,utc,lc,iq,lat1,lon1,lat2,lon2,nb_mes,big_nb_mes,best_level,pass_duration,nopc,calcul_freq,altitude
0,20616,29051,7/3/2003 9:13,3,66,33.898,-77.958,27.369,-46.309,6,0,-126,529,3,401 651134.7,0
1,21892,29051,7/3/2003 9:23,B,0,33.887,-77.950,43.983,-128.418,2,0,-132,96,1,401 651133.1,0
2,20618,29051,7/3/2003 10:31,A,6,33.884,-77.946,22.875,-25.276,3,0,-132,320,3,401 651169.3,0
3,20619,29051,7/3/2003 10:49,A,7,33.927,-78.008,37.291,-94.435,3,0,-125,260,2,401 651166.8,0
4,20620,29051,7/3/2003 12:10,2,67,33.901,-77.909,31.005,-64.614,4,0,-124,612,3,401 651181.6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,287868,29051,3/23/2005 18:47,0,50,33.314,-77.616,37.216,-59.366,4,0,-125,152,1,401 651107.1,0
2492,287869,29051,3/23/2005 19:55,B,0,33.315,-77.636,42.555,-30.409,2,0,-134,49,1,401 651113.6,0
2493,287870,29051,3/23/2005 22:47,B,0,33.270,-77.611,35.079,-72.205,2,0,-130,49,1,401 651113.6,0
2494,287872,29051,3/24/2005 2:44,B,0,33.353,-77.628,35.949,-64.804,2,0,-122,55,1,401 651113.6,0


In [6]:
#Create a mask of user dates
dateMask = df['utc'].str.startswith(userDate)

In [7]:
#Create a mask of location classe
lcMask = df['lc'].isin(locClasses)

In [8]:
#Filter the records that match the above masks
dfOutput = df[dateMask & lcMask]
dfOutput.head()

,uid,tag_id,utc,lc,iq,lat1,lon1,lat2,lon2,nb_mes,big_nb_mes,best_level,pass_duration,nopc,calcul_freq,altitude
0,20616,29051,7/3/2003 9:13,3,66,33.898,-77.958,27.369,-46.309,6,0,-126,529,3,401 651134.7,0
4,20620,29051,7/3/2003 12:10,2,67,33.901,-77.909,31.005,-64.614,4,0,-124,612,3,401 651181.6,0


In [9]:
print ("On {}, Sara the turtle was found at:".format(userDate))
for i,row in dfOutput.iterrows():
    print('  {0:.3f}° Lat; {1:.3f}° Lon'.format(row['lat1'],row['lon1']))

On 7/3/2003, Sara the turtle was found at:
  33.898° Lat; -77.958° Lon
  33.901° Lat; -77.909° Lon


## Mapping the points with the ArcGIS Python API

In [10]:
#Create the GIS object
from arcgis import GIS, geometry
gis = GIS()

In [11]:
#Create the map object
m = gis.map()

#Loop through selected records
for i,row in dfOutput.iterrows():
    
    #Create a point, projected to WGS84
    thePoint = geometry.Point({"x":row['lon1'],
                               "y":row['lat1'],
                               "spatialRefernce":{'wkid':'4326'}
                              })
    
    #Draw that point in the map, assigning a popup to that point
    m.draw(thePoint,popup={'title':str(i),
                           'content':row['utc']})
    
    #Center the map on the current point
    m.center = [row['lat1'],row['lon1']]
    
#Display the map at zoom level 7, using oceans basemap
m.zoom = 7
m.basemap='oceans'
m

MapView(layout=Layout(height='400px', width='100%'))